# **Preprocesamiento de nombres de directivos**

In [92]:
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
import dotenv
import os
from pydantic import BaseModel
import unicodedata
import re
from typing import Optional

### **Carga de datos**

In [93]:
dotenv.load_dotenv()
API_KEY = os.getenv("OPENAI_API_KEY")

In [94]:
df = pd.read_csv('data/directors_and_companies.csv')
df.head()

,Rut_individuo,Nombre,Cargo,Fecha_nombramiento,Fecha_termino,Rut_Empresa,Nombre_empresa
0,0-E,LUIS ORVAÑANOS LASCURAIN,Presidente,28/03/2012,NaN,59175530,"CORPORACION GEO, S.A.B. DE C.V."
1,0-E,EMILIO CUENCA FRIEDERICHSEN,Director,28/03/2012,NaN,59175530,"CORPORACION GEO, S.A.B. DE C.V."
2,0-E,ROBERTO ORVAÑANOS CONDE,Director,28/03/2012,NaN,59175530,"CORPORACION GEO, S.A.B. DE C.V."
3,0-E,IÑIGO ORVAÑANOS CORCUERA,Director,28/03/2012,NaN,59175530,"CORPORACION GEO, S.A.B. DE C.V."
4,0-E,RAUL ZORRILLA COSIO,Director,28/03/2012,NaN,59175530,"CORPORACION GEO, S.A.B. DE C.V."


### **Limpiamos los nombres de los directivos**

In [95]:
def clean_name(name: str):
    capital_name = name.strip().upper()
    clean_name = unicodedata.normalize('NFD', capital_name).encode('ascii', 'ignore').decode('utf-8')
    clean_name = re.sub(r'[^a-zA-Z0-9\s]', '', clean_name)
    return clean_name

df['Nombre'] = df['Nombre'].apply(clean_name)
df.head()

,Rut_individuo,Nombre,Cargo,Fecha_nombramiento,Fecha_termino,Rut_Empresa,Nombre_empresa
0,0-E,LUIS ORVANANOS LASCURAIN,Presidente,28/03/2012,NaN,59175530,"CORPORACION GEO, S.A.B. DE C.V."
1,0-E,EMILIO CUENCA FRIEDERICHSEN,Director,28/03/2012,NaN,59175530,"CORPORACION GEO, S.A.B. DE C.V."
2,0-E,ROBERTO ORVANANOS CONDE,Director,28/03/2012,NaN,59175530,"CORPORACION GEO, S.A.B. DE C.V."
3,0-E,INIGO ORVANANOS CORCUERA,Director,28/03/2012,NaN,59175530,"CORPORACION GEO, S.A.B. DE C.V."
4,0-E,RAUL ZORRILLA COSIO,Director,28/03/2012,NaN,59175530,"CORPORACION GEO, S.A.B. DE C.V."


### **Eliminar duplicados**

In [96]:
company_names_df = df[['Nombre', 'Nombre_empresa']].dropna(subset=['Nombre_empresa', 'Nombre'])
company_names_df.shape

(15841, 2)

In [97]:
company_names_df_without_duplicates = company_names_df.drop_duplicates()
print(f"Shape without duplicates: {company_names_df_without_duplicates.shape}")
company_names_df_without_duplicates.head()

Shape without duplicates: (6946, 2)


,Nombre,Nombre_empresa
0,LUIS ORVANANOS LASCURAIN,"CORPORACION GEO, S.A.B. DE C.V."
1,EMILIO CUENCA FRIEDERICHSEN,"CORPORACION GEO, S.A.B. DE C.V."
2,ROBERTO ORVANANOS CONDE,"CORPORACION GEO, S.A.B. DE C.V."
3,INIGO ORVANANOS CORCUERA,"CORPORACION GEO, S.A.B. DE C.V."
4,RAUL ZORRILLA COSIO,"CORPORACION GEO, S.A.B. DE C.V."


### **Separamos los nombres y apellidos de los directivos**

In [118]:
class NombreDirectivo(BaseModel):
    """
    Nombre de un directivo
    """
    nombres: str
    primer_apellido: str
    segundo_apellido: str

In [119]:
examples = [
    {
        "input": "Separa el nombre y los apellidos de: Lucas Carrasco Estay",
        "output": {"nombres": "Lucas", "primer_apellido": "Carrasco", "segundo_apellido": "Estay"}
    },
    {
        "input": "Separa el nombre y los apellidos de: Juan Pablo González",
        "output": {"nombres": "Juan Pablo", "primer_apellido": "González", "segundo_apellido": ""}
    },
    {
        "input": "Separa el nombre y los apellidos de: Maria Garcia Fernandez de Leon",
        "output": {"nombres": "Maria", "primer_apellido": "Garcia", "segundo_apellido": "Fernandez de Leon"}
    },
    {
        "input": "Separa el nombre y los apellidos de: Juan Carlos de la Torre",
        "output": {"nombres": "Juan Carlos", "primer_apellido": "de la Torre", "segundo_apellido": ""}
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [120]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Eres un experto en separar nombres y apellidos."),
        few_shot_prompt,
        ("user", "Separa el nombre y los apellidos de: {name}"),
    ]
)

In [121]:
model = ChatOpenAI(
    openai_api_key=API_KEY, 
    model="gpt-4o-mini",
)
structured_llm = model.with_structured_output(NombreDirectivo)

chain = prompt | structured_llm
chain

ChatPromptTemplate(input_variables=['name'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Eres un experto en separar nombres y apellidos.')), FewShotChatMessagePromptTemplate(examples=[{'input': 'Separa el nombre y los apellidos de: Lucas Carrasco Estay', 'output': {'nombres': 'Lucas', 'primer_apellido': 'Carrasco', 'segundo_apellido': 'Estay'}}, {'input': 'Separa el nombre y los apellidos de: Juan Pablo González', 'output': {'nombres': 'Juan Pablo', 'primer_apellido': 'González', 'segundo_apellido': ''}}, {'input': 'Separa el nombre y los apellidos de: Maria Garcia Fernandez de Leon', 'output': {'nombres': 'Maria', 'primer_apellido': 'Garcia', 'segundo_apellido': 'Fernandez de Leon'}}, {'input': 'Separa el nombre y los apellidos de: Juan Carlos de la Torre', 'output': {'nombres': 'Juan Carlos', 'primer_apellido': 'de la Torre', 'segundo_apellido': ''}}], example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], messages=[HumanM

In [123]:
nombres = []
primer_apellidos = []
segundo_apellidos = []
for index, row in company_names_df_without_duplicates.iterrows():
    response = chain.invoke({"name": row['Nombre']})
    print(f"Nombre: {row['Nombre']}, Response: {response}")
    nombres.append(response['nombres'])
    primer_apellidos.append(response['primer_apellido'])
    segundo_apellidos.append(response['segundo_apellido'])
    
company_names_df_without_duplicates['Nombres'] = nombres
company_names_df_without_duplicates['Primer_apellido'] = primer_apellidos
company_names_df_without_duplicates['Segundo_apellido'] = segundo_apellidos
company_names_df_without_duplicates.head()

Nombre: LUIS ORVANANOS LASCURAIN, Response: {'nombres': 'LUIS', 'primer_apellido': 'ORVANANOS', 'segundo_apellido': 'LASCURAIN'}
Nombre: EMILIO CUENCA FRIEDERICHSEN, Response: {'nombres': 'EMILIO', 'primer_apellido': 'CUENCA', 'segundo_apellido': 'FRIEDERICHSEN'}
Nombre: ROBERTO ORVANANOS CONDE, Response: {'nombres': 'ROBERTO', 'primer_apellido': 'ORVANANOS', 'segundo_apellido': 'CONDE'}
Nombre: INIGO ORVANANOS CORCUERA, Response: {'nombres': 'INIGO', 'primer_apellido': 'ORVANANOS', 'segundo_apellido': 'CORCUERA'}
Nombre: RAUL ZORRILLA COSIO, Response: {'nombres': 'RAUL', 'primer_apellido': 'ZORRILLA', 'segundo_apellido': 'COSIO'}
Nombre: LUIS IGNACIO ADJELJALEK MARTINEZ, Response: {'nombres': 'LUIS IGNACIO', 'primer_apellido': 'ADJELJALEK', 'segundo_apellido': 'MARTINEZ'}
Nombre: ANDRES CAIRE OBREGON, Response: {'nombres': 'ANDRES', 'primer_apellido': 'CAIRE', 'segundo_apellido': 'OBREGON'}
Nombre: GABRIEL GOMEZ CASTANARES, Response: {'nombres': 'GABRIEL', 'primer_apellido': 'GOMEZ', 

C:\Users\lucas\AppData\Local\Temp\ipykernel_12676\3271963171.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  company_names_df_without_duplicates['Nombres'] = nombres
C:\Users\lucas\AppData\Local\Temp\ipykernel_12676\3271963171.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  company_names_df_without_duplicates['Primer_apellido'] = primer_apellidos
C:\Users\lucas\AppData\Local\Temp\ipykernel_12676\3271963171.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

,Nombre,Nombre_empresa,Nombres,Primer_apellido,Segundo_apellido
0,LUIS ORVANANOS LASCURAIN,"CORPORACION GEO, S.A.B. DE C.V.",LUIS,ORVANANOS,LASCURAIN
1,EMILIO CUENCA FRIEDERICHSEN,"CORPORACION GEO, S.A.B. DE C.V.",EMILIO,CUENCA,FRIEDERICHSEN
2,ROBERTO ORVANANOS CONDE,"CORPORACION GEO, S.A.B. DE C.V.",ROBERTO,ORVANANOS,CONDE
3,INIGO ORVANANOS CORCUERA,"CORPORACION GEO, S.A.B. DE C.V.",INIGO,ORVANANOS,CORCUERA
4,RAUL ZORRILLA COSIO,"CORPORACION GEO, S.A.B. DE C.V.",RAUL,ZORRILLA,COSIO


In [124]:
company_names_df_without_duplicates.to_csv('data/directors_and_companies_separated.csv', index=False)